create simple model draft to get outlet embeddings 

In [187]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch 
from torchmetrics.functional.classification import f1_score
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F
from datasets import Dataset
from transformers.optimization import get_linear_schedule_with_warmup
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm 
import random
from sklearn.preprocessing import OneHotEncoder

In [2]:
# load in the news data of interest 
df = pd.read_csv("/shared/3/projects/newsDiffusion/data/processed/fullDataWithClustNums.tsv", sep="\t") 

<ipython-input-2-e2098ce75a41>:2: DtypeWarning: Columns (2,3,10,11,12,13,14,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/shared/3/projects/newsDiffusion/data/processed/fullDataWithClustNums.tsv", sep="\t")


In [5]:
len(df["clustNum"].unique()) 

209415

In [6]:
embeddingsDf = pd.read_csv("/shared/3/projects/newsDiffusion/data/interim/NEREmbedding/embeddingsKeys.tsv", sep="\t", names=["key", "embedding"], converters={"embedding":lambda x: np.array(x.strip("[]").split(","), dtype=float)})

In [7]:
#we want to get a matrix of outlets x stories 
storyEmbeddings = pd.merge(df[["clustNum", "key"]].dropna(), embeddingsDf, on="key", how="left") 
storyEmbeddings = storyEmbeddings.drop(columns=["key"]).groupby("clustNum").agg([np.mean, len])
storyEmbeddings.columns = storyEmbeddings.columns.droplevel()
storyEmbeddings = storyEmbeddings[storyEmbeddings["len"] < 3000].reset_index() 
storyEmbeddings = storyEmbeddings.rename(columns={"mean":"storyMean", "len":"clustSize"}) 

In [ ]:
PICKLE_PATH = "/shared/3/projects/newsDiffusion/data/processed/IRTmodel/storyEmbeddingsMean.pkl"
storyEmbeddings.to_pickle(PICKLE_PATH)